## Data Cleaning for Home Mortgage Dataset

### Load Raw Dataset

In [1]:
# import dependencies
import pandas as pd
from path import Path 
import numpy as np 
import os
from random import randint

In [2]:
# set path and load dataframe
path = Path('../data/hmda_2017_ca_all-records_labels.csv')
df = pd.read_csv(path, low_memory=False)
df.head()

,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home improvement,...,NaN,NaN,NaN,4824.0,37.230000,75200.0,57.419998,818.0,1626.0,NaN
1,2017,0000146672,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,7404.0,57.520000,63200.0,116.010002,1215.0,1743.0,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,3372.0,33.189999,97400.0,141.740005,592.0,1105.0,NaN
3,2017,13-6131491,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,8787.0,65.129997,97400.0,97.269997,1463.0,2164.0,NaN
4,2017,33-0419992,Department of Housing and Urban Development,HUD,7,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,5356.0,23.100000,75200.0,126.690002,1711.0,2102.0,NaN


In [3]:
df = df.dropna(how='all')
df_info = df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714459 entries, 0 to 1714458
Data columns (total 78 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   as_of_year                      int64  
 1   respondent_id                   object 
 2   agency_name                     object 
 3   agency_abbr                     object 
 4   agency_code                     int64  
 5   loan_type_name                  object 
 6   loan_type                       int64  
 7   property_type_name              object 
 8   property_type                   int64  
 9   loan_purpose_name               object 
 10  loan_purpose                    int64  
 11  owner_occupancy_name            object 
 12  owner_occupancy                 int64  
 13  loan_amount_000s                float64
 14  preapproval_name                object 
 15  preapproval                     int64  
 16  action_taken_name               object 
 17  action_taken               

In [4]:
df.duplicated().sum()

4957

In [5]:
df = df.drop_duplicates(ignore_index=True)
df.head()

,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home improvement,...,NaN,NaN,NaN,4824.0,37.230000,75200.0,57.419998,818.0,1626.0,NaN
1,2017,0000146672,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,7404.0,57.520000,63200.0,116.010002,1215.0,1743.0,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,NaN,NaN,NaN,3372.0,33.189999,97400.0,141.740005,592.0,1105.0,NaN
3,2017,13-6131491,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,8787.0,65.129997,97400.0,97.269997,1463.0,2164.0,NaN
4,2017,33-0419992,Department of Housing and Urban Development,HUD,7,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Refinancing,...,NaN,NaN,NaN,5356.0,23.100000,75200.0,126.690002,1711.0,2102.0,NaN


In [6]:
row_count = len(df)
print(f'Row Count: {row_count}')

Row Count: 1709502


In [7]:
column_count = len(df.columns)
print(f'Column Count: {column_count}')

Column Count: 78


Add tracking information to sequence_number, edit_status, edit_status_name

In [8]:
# define random_int function to generate a random ID number for each application, with some number of digits
def random_int(n):
    range_start = 10**(n-1)+1
    range_end = 10**(n)-1
    return randint(range_start, range_end)

In [9]:
def assign_id_num(member):
    member = random_int(12)
    return member

In [10]:
df['sequence_number'] = df['sequence_number'].apply(assign_id_num)
df['sequence_number'][:-5]

0          191856311741
1          798333441520
2          172228252986
3          723494359341
4          503311073815
               ...     
1709492    486136353233
1709493    431389165352
1709494    136602720132
1709495    949053397920
1709496    267082181285
Name: sequence_number, Length: 1709497, dtype: int64

In [11]:
def check_processing_status(data):
    status_name = data['edit_status_name'].unique()
    status_level = data['edit_status'].unique()
    print(f'Status: {status_name} | Level {status_level}')


In [12]:
status_log_dict = {}

def update_status(data, dict, status, level):
    data.copy() 

    status = status
    level = int(level)

    dict[status] = [level]

    data['edit_status_name'] = status
    data['edit_status'] = level

    return data 

In [13]:
status = 'import'
level = 0
df = update_status(df, status_log_dict, status, level)
status_log_dict

{'import': [0]}

In [14]:
status = 'preprocessing'
level = 1
df = update_status(df, status_log_dict, status, level)

In [15]:
# export prelimarily cleaned data
df.to_csv('../data/cleaned_data.csv', index=False, encoding='utf8')

In [16]:
df.head()

,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home improvement,...,preprocessing,1,191856311741,4824.0,37.230000,75200.0,57.419998,818.0,1626.0,NaN
1,2017,0000146672,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,preprocessing,1,798333441520,7404.0,57.520000,63200.0,116.010002,1215.0,1743.0,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,preprocessing,1,172228252986,3372.0,33.189999,97400.0,141.740005,592.0,1105.0,NaN
3,2017,13-6131491,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,preprocessing,1,723494359341,8787.0,65.129997,97400.0,97.269997,1463.0,2164.0,NaN
4,2017,33-0419992,Department of Housing and Urban Development,HUD,7,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Refinancing,...,preprocessing,1,503311073815,5356.0,23.100000,75200.0,126.690002,1711.0,2102.0,NaN


#### Extract all columns

In [17]:
df_columns = df.columns
print(f'Row Count: {len(df)} | Column Count: {len(df_columns)}')
print(df_columns)


Row Count: 1709502 | Column Count: 78
Index(['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr',
       'agency_code', 'loan_type_name', 'loan_type', 'property_type_name',
       'property_type', 'loan_purpose_name', 'loan_purpose',
       'owner_occupancy_name', 'owner_occupancy', 'loan_amount_000s',
       'preapproval_name', 'preapproval', 'action_taken_name', 'action_taken',
       'msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code',
       'county_name', 'county_code', 'census_tract_number',
       'applicant_ethnicity_name', 'applicant_ethnicity',
       'co_applicant_ethnicity_name', 'co_applicant_ethnicity',
       'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2',
       'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3',
       'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5',
       'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1',
       'co_applicant_race_name_2', 'co_applicant_ra

#### Divide the dataset by column_type

In [18]:
# constant, float
tracking_columns = ['edit_status_name', 'edit_status', 'sequence_number']

#demographic float columns, 
demographics_columns = ['population', 'minority_population', 'hud_median_family_income', 'tract_to_msamd_income', 'number_of_owner_occupied_units', 'number_of_1_to_4_family_units']

# categorical text/numeric
applicant_sex_columns = ['applicant_sex_name', 'applicant_sex', 'co_applicant_sex_name', 'co_applicant_sex']

race_ethnicity_columns = ['applicant_ethnicity_name', 'applicant_ethnicity', 'co_applicant_ethnicity_name', 'co_applicant_ethnicity', 'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2', 'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3', 'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5', 'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1', 'co_applicant_race_name_2', 'co_applicant_race_2', 'co_applicant_race_name_3', 'co_applicant_race_3', 'co_applicant_race_name_4', 'co_applicant_race_4', 'co_applicant_race_name_5', 'co_applicant_race_5']

# loan info float
monetary_columns = ['loan_amount_000s', 'applicant_income_000s','rate_spread']

# loan info categorical
loan_info_columns = ['loan_type_name', 'loan_type', 'property_type_name', 'property_type', 'loan_purpose_name', 'loan_purpose', 'owner_occupancy_name', 'owner_occupancy', 'purchaser_type_name', 'purchaser_type','hoepa_status_name', 'hoepa_status', 'lien_status_name',
'lien_status']

# pre/approval
preapproval_columns = ['preapproval_name', 'preapproval']
approval_columns = ['action_taken_name', 'action_taken']

# denial
denial_reason_columns = ['denial_reason_name_1', 'denial_reason_1', 'denial_reason_name_2','denial_reason_2', 'denial_reason_name_3', 'denial_reason_3']

# date/time/location columns
date_time_loc_columns = ['as_of_year', 'respondent_id', 'agency_name', 'agency_abbr', 'agency_code','msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code', 'county_name', 'county_code', 'census_tract_number', 'application_date_indicator']


In [19]:
# compile all column types into column_types_dict
column_types_dict = {'tracking_columns': tracking_columns, 
                    'demographics_columns': demographics_columns, 
                    'applicant_sex_columns' : applicant_sex_columns,
                    'race_ethnicity_columns' : race_ethnicity_columns,
                    'monetary_columns' : monetary_columns,
                    'loan_info_columns' : loan_info_columns,
                    'preapproval_columns' : preapproval_columns,
                    'approval_columns' : approval_columns,
                    'denial_reason_columns' : denial_reason_columns,      
                    'date_time_loc_columns' : date_time_loc_columns}

In [20]:
# check the len of column_types_dict to make sure we didn't lose any columns
def count_column_types(column_dict):
    # initialize counter
    counter = 0
    # loop through each value in dictionary
    for value in column_dict.values():
        value_count = len(value)
        counter += value_count

    return counter

In [21]:
print(f'Total Column Count: {count_column_types(column_types_dict)}')
print(f'Column Types: {list(column_types_dict.keys())}')
print(f'Column Type Count: {len(column_types_dict.values())}')


# All good!

Total Column Count: 78
Column Types: ['tracking_columns', 'demographics_columns', 'applicant_sex_columns', 'race_ethnicity_columns', 'monetary_columns', 'loan_info_columns', 'preapproval_columns', 'approval_columns', 'denial_reason_columns', 'date_time_loc_columns']
Column Type Count: 10
